# Integração com APIs: Feriados e Tempo

Utilize as APIs públicas abaixo para responder às questões 1-8:

Instalando Bibliotecas

In [ ]:
pip install openmeteo-requests

In [ ]:
pip install requests-cache

In [ ]:
pip install retry-requests

In [ ]:
# Importando Bibliotecas

import requests
import pandas as pd
from datetime import datetime as dt
import statistics
from collections import Counter
import json
import openmeteo_requests
import requests_cache
from retry_requests import retry

In [ ]:
# Função para acessar API Holiday

def feriados(Year, CountryCode):

    url = f"https://date.nager.at/api/v3/PublicHolidays/{Year}/{CountryCode}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return f"Erro: {response.status_code}"

In [ ]:
# Chamando função feriados com ano atual e código do Brasil

feriados = feriados(2024, 'BR')

Código utilizado para acesso à API: [Open-Meteo](https://open-meteo.com/en/docs/historical-weather-api#latitude=-22.9064&longitude=-43.1822&start_date=2024-01-01&end_date=2024-08-01&hourly=&daily=weather_code,temperature_2m_mean&timezone=America%2FSao_Paulo)




In [ ]:
# Acessar API Open Meteo

cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": -22.9064,
	"longitude": -43.1822,
	"start_date": "2024-01-01",
	"end_date": "2024-08-01",
	"daily": ["weather_code", "temperature_2m_mean"],
	"timezone": "America/Sao_Paulo"
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
response.Latitude()
response.Longitude()
response.Elevation()
response.Timezone()
response.TimezoneAbbreviation()
response.UtcOffsetSeconds()

daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean

daily_dataframe = pd.DataFrame(data = daily_data)

## 1. Quantos feriados há no Brasil em todo o ano de 2024?

In [ ]:
count = 0

for feriado in feriados:
  count += 1

print(f'Há {count} feriados no Brasil no ano de 2024')

Há 14 feriados no Brasil no ano de 2024


## 2. Qual mês de 2024 tem o maior número de feriados?

In [ ]:
mes = []

for feriado in feriados:
  mes.append(dt.strptime(feriado['date'], '%Y-%m-%d').month)

mes_com_mais_feriados, numero_de_feriados = Counter(mes).most_common(1)[0]
print(f"O mês do ano com o maior número de feriados em 2024 é o {mes_com_mais_feriados}º mes, com {numero_de_feriados} feriados.")

O mês do ano com o maior número de feriados em 2024 é o 2º mes, com 2 feriados.


## 3. Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?

In [ ]:
semana = []

for feriado in feriados:
  if dt.strptime(feriado['date'], '%Y-%m-%d').weekday() < 5:
    semana.append(feriado['date'])

print(f"O número de feriados em 2024 que caem de segunda a sexta é {len(semana)}")

O número de feriados em 2024 que caem de segunda a sexta é 9


## 4. Qual foi a temperatura média em cada mês?
Utilize a Open-Meteo Historical Weather API para obter as temperaturas médias diárias no Rio de Janeiro de 01/01/2024 a 01/08/2024.

In [ ]:
df_mensal = daily_dataframe.groupby(daily_dataframe['date'].dt.to_period('M')).mean()
print(f'Segue na tabela a baixo a temperatura média em cada mês no periodo determinado:')
df_mensal['temperature_2m_mean']

Segue na tabela a baixo a temperatura média em cada mês no periodo determinado:


,temperature_2m_mean
date,
2024-01,26.694233
2024-02,27.212988
2024-03,26.416479
2024-04,25.169930
2024-05,24.990740
2024-06,22.648958
2024-07,21.105591
2024-08,21.225414


## 5. Qual foi o tempo predominante em cada mês nesse período?
Utilize como referência para o código de tempo (weather_code) o seguinte link: WMO Code.

In [ ]:
with open('/content/descriptions.json', 'r') as f:
    weather_code_description = json.load(f)

In [ ]:
codes = []
day = []
night = []

for code, info in weather_code_description.items():
    codes.append(code)
    day.append(info['day']['description'])
    night.append(info['night']['description'])


df_code = pd.DataFrame({
    'Codigos': codes,
    'Dia': day,
    'Noite': night,
})

In [ ]:
daily_dataframe['weather_code'] = daily_dataframe['weather_code'].astype(int)
df_code['Codigos'] = df_code['Codigos'].astype(int)

In [ ]:
df_code_mes = df_code.merge(daily_dataframe, left_on='Codigos', right_on='weather_code')

In [ ]:
df_code_mes = df_code_mes.groupby(df_code_mes['date'].dt.to_period('M')).agg({'Dia': lambda x: x.mode()[0],'Noite': lambda x: x.mode()[0],'weather_code': lambda x: x.mode()[0], 'temperature_2m_mean': 'mean'})[['Dia', 'Noite', 'weather_code','temperature_2m_mean']]

<ipython-input-16-363445b2e81f>:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_code_mes = df_code_mes.groupby(df_code_mes['date'].dt.to_period('M')).agg({'Dia': lambda x: x.mode()[0],'Noite': lambda x: x.mode()[0],'weather_code': lambda x: x.mode()[0], 'temperature_2m_mean': 'mean'})[['Dia', 'Noite', 'weather_code','temperature_2m_mean']]


In [ ]:
print(f'Segue na tabela a baixo o tempo predominante em cada mês no periodo determinado:')
df_code_mes

Segue na tabela a baixo o tempo predominante em cada mês no periodo determinado:


,Dia,Noite,weather_code,temperature_2m_mean
date,,,,
2024-01,Rain,Rain,63,26.694233
2024-02,Light Drizzle,Light Drizzle,51,27.212988
2024-03,Light Drizzle,Light Drizzle,51,26.416479
2024-04,Cloudy,Cloudy,3,25.169930
2024-05,Light Drizzle,Light Drizzle,51,24.990740
2024-06,Mainly Sunny,Mainly Clear,1,22.648958
2024-07,Cloudy,Cloudy,3,21.105591
2024-08,Partly Cloudy,Partly Cloudy,2,21.225414


## 6. Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?

In [ ]:
mes = []
nome = []

for feriado in feriados:
  mes.append(feriado['date'])
  nome.append(feriado['localName'])

df_feriados = pd.DataFrame({"Data Feriado": mes , "Feriado": nome })

df_feriados['Data Feriado'] = pd.to_datetime(df_feriados['Data Feriado'])

In [ ]:
daily_dataframe['date'] = daily_dataframe['date'].dt.strftime('%Y-%m-%d')

In [ ]:
df_feriados['Data Feriado'] = pd.to_datetime(df_feriados['Data Feriado'])
daily_dataframe['date'] = pd.to_datetime(daily_dataframe['date'])

In [ ]:
df_feriados = daily_dataframe.merge(df_feriados, left_on='date', right_on='Data Feriado')

In [ ]:
df_feriados = df_feriados.merge(df_code, left_on='weather_code', right_on='Codigos')

In [ ]:
print(f'Segue na tabela a baixo o tempo e a temperatura média em cada feriado no periodo determinado:')
df_feriados[['Data Feriado', 'Feriado', 'temperature_2m_mean', 'Dia', 'Noite', 'Codigos']]

Segue na tabela a baixo o tempo e a temperatura média em cada feriado no periodo determinado:


,Data Feriado,Feriado,temperature_2m_mean,Dia,Noite,Codigos
0,2024-01-01,Confraternização Universal,24.854582,Light Drizzle,Light Drizzle,51
1,2024-02-13,Carnaval,30.531670,Light Drizzle,Light Drizzle,51
2,2024-05-30,Corpus Christi,20.779579,Light Drizzle,Light Drizzle,51
3,2024-02-12,Carnaval,30.156668,Mainly Sunny,Mainly Clear,1
4,2024-03-29,Sexta-feira Santa,25.044168,Light Rain,Light Rain,61
5,2024-03-31,Domingo de Páscoa,24.531664,Drizzle,Drizzle,53
6,2024-07-09,Revolução Constitucionalista de 1932,21.487915,Drizzle,Drizzle,53
7,2024-04-21,Dia de Tiradentes,23.087919,Partly Cloudy,Partly Cloudy,2
8,2024-05-01,Dia do Trabalhador,28.108751,Sunny,Clear,0


## 7. Considere as seguintes suposições:


*  O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
* Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
* O cidadão carioca só vai à praia quando não está com frio;
* O cidadão carioca também só vai à praia em dias com sol, evitando dias totalmente nublados ou chuvosos (considere weather_code para determinar as condições climáticas).

Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?

In [ ]:
df_code['Dia']

,Dia
0,Sunny
1,Mainly Sunny
2,Partly Cloudy
3,Cloudy
4,Foggy
5,Rime Fog
6,Light Drizzle
7,Drizzle
8,Heavy Drizzle
9,Light Freezing Drizzle


In [ ]:
print(f'Segue na tabela a baixo os feriados não aproveitávies para o carioca no periodo determinado:')
df_nao_aproveitavel = df_feriados[(df_feriados['temperature_2m_mean'] < 20) | (~df_feriados['Dia'].isin(['Sunny', 'Mainly Sunny', 'Partly Cloudy']))]
df_nao_aproveitavel[['date', 'Feriado']]

Segue na tabela a baixo os feriados não aproveitávies para o carioca no periodo determinado:
Os feriados são: Confraternização Universal, Carnaval, Corpus Christi, Sexta-feira Santa, Domingo de Páscoa, Revolução Constitucionalista de 1932


,date,Feriado
0,2024-01-01,Confraternização Universal
1,2024-02-13,Carnaval
2,2024-05-30,Corpus Christi
4,2024-03-29,Sexta-feira Santa
5,2024-03-31,Domingo de Páscoa
6,2024-07-09,Revolução Constitucionalista de 1932


Os feriados são: Confraternização Universal, Carnaval, Corpus Christi, Sexta-feira Santa, Domingo de Páscoa, Revolução Constitucionalista de 1932

## 8. Qual foi o feriado "mais aproveitável" de 2024?
Considere o melhor par tempo e temperatura.

In [ ]:
df_aproveitavel = df_feriados[(df_feriados['temperature_2m_mean'] >= 20) & (df_feriados['Dia'].isin(['Sunny', 'Mainly Sunny', 'Partly Cloudy']))]

In [ ]:
df_aproveitavel[['date', 'Feriado', 'temperature_2m_mean', 'Dia', 'Noite']]

,date,Feriado,temperature_2m_mean,Dia,Noite
3,2024-02-12,Carnaval,30.156668,Mainly Sunny,Mainly Clear
7,2024-04-21,Dia de Tiradentes,23.087919,Partly Cloudy,Partly Cloudy
8,2024-05-01,Dia do Trabalhador,28.108751,Sunny,Clear


Dependendo da prioridade, seja a temperatura mais alta ou o dia mais ensolarado, os feriados de **Carnaval** ou **Dia do Trabalhador**, respectivamente, são os mais aproveitável para o carioca no ano de 2024